### Import Libraries

In [47]:
import re
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
%matplotlib inline

### Install Watermark - tool to help with reproducibility:

In [17]:
%install_ext https://raw.githubusercontent.com/rasbt/watermark/master/watermark/watermark.py

Installed watermark.py. To use it, type:
  %load_ext watermark


In [2]:
%load_ext watermark
%watermark -n -t -z -u -m -v -p matplotlib,numpy,conda

last updated: Fri Jun 24 2016 09:44:36 CDT

CPython 2.7.11
IPython 4.0.3

matplotlib 1.5.1
numpy 1.10.1
conda 4.0.8

compiler   : GCC 4.2.1 (Apple Inc. build 5577)
system     : Darwin
release    : 15.5.0
machine    : x86_64
processor  : i386
CPU cores  : 4
interpreter: 64bit


In [7]:
file_name = '/Users/elisa/Documents/CompLing/compSemantics/HW3/wikicorpus.txt'
with open(file_name, 'r') as f:
    lines = f.readlines()

In [ ]:
all_words = []
for line in lines:
    match = re.search(r'^<c> ', line)
    if match:
        all_words.extend([word for word in line[match.end(0):].split(" ")])
filtered_words = [word for word in all_words if word not in stopwords.words('english')]
print filtered_words[:2]

AttributeError: 'NoneType' object has no attribute 'group'